# Occurrence data for Invertebrate Paleo Group Project, Spring 2021, AMNH RGGS

This Jupyter notebook accesses data from the PaleoBioDB directly using URLs formatted freom the paleobiodb.org website.
You should run what's in here already to see how it works before modifying for your own uses.

This notebook is set up to create 3 CSV files, one each for the Kaili, Poleta, and Burgess Shale formations. It collates id names
from columns in the DB and also makes a list of references from the database.

*You can hit "run all" but it will take a minute or two.*

## Load libraries.

Some of this stuff is vestigial tails from other projects
(for example, matplotlib isn't necessary but I'm leaving it in here out of habit).

In [2]:
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib inline

# PBDB Queries

Load in PBDB occurrences from Asia and North America, from 520-500mya. These URLs are from the "Download Records"
page on paleobiodb.org.

The CSV files are cached in the filenames specified below. You can force downloading by either deleting the CSV files
or by setting FORCE_DOWNLOAD below to True.

*Note that the data download may take a minute or two.* Watch for the ampersand to turn to a number in the brackets for the cell, indicating that the cell is finished running.

In [3]:
FORCE_DOWNLOAD = False

asia_520_500_pbdb_url = (
    "http://paleobiodb.org/data1.2/occs/list.csv?datainfo&rowcount&max_ma=520&min_ma=500&cc=ASI&"
    "lithology=siliciclastic,mixed,carbonate,evaporite,organic,chemical,volcanic,metasedimentary,"
    "metamorphic,other,unknown&envtype=terr,marine,carbonate,silicic,unknown,lacust,fluvial,karst,"
    "terrother,marginal,reef,stshallow,stdeep,offshore,slope,marindet&show=full,genus,subgenus,strat,env,ref"
)
asia_csv_filename = "pbdb_asia_raw_520_500.csv"
    
na_520_500_pbdb_url = (
    "http://paleobiodb.org/data1.2/occs/list.csv?datainfo&rowcount&max_ma=520&min_ma=500&cc=NOA&"
    "lithology=siliciclastic,mixed,carbonate,evaporite,organic,chemical,volcanic,metasedimentary,"
    "metamorphic,other,unknown&envtype=terr,marine,carbonate,silicic,unknown,lacust,fluvial,karst,"
    "terrother,marginal,reef,stshallow,stdeep,offshore,slope,marindet&show=full,genus,subgenus,strat,env,ref"
)
na_csv_filename = "pbdb_na_raw_520_500.csv"

if (not os.path.exists(asia_csv_filename)) or FORCE_DOWNLOAD == True:
    r = requests.get(asia_520_500_pbdb_url, allow_redirects=True)
    open(asia_csv_filename, 'wb').write(r.content)

if (not os.path.exists(na_csv_filename)) or FORCE_DOWNLOAD == True:
    r = requests.get(na_520_500_pbdb_url, allow_redirects=True)
    open(na_csv_filename, 'wb').write(r.content)

# Kaili.
pbdb_asia_occs_df = pd.read_csv(asia_csv_filename, 
                                low_memory=False, 
                                encoding='latin1',
                                skiprows=21)
kaili_occs_df = pbdb_asia_occs_df[pbdb_asia_occs_df.formation == 'Kaili']

# Poleta.
pbdb_na_occs_df = pd.read_csv(na_csv_filename, 
                              low_memory=False,
                              encoding='latin1',
                              skiprows=21)
poleta_occs_df = pbdb_na_occs_df[pbdb_na_occs_df.formation == 'Poleta']

# Burgess Shale.
burgess_occs_df = pbdb_na_occs_df[pbdb_na_occs_df.formation == 'Burgess Shale']

There are lots of columns. Like, *lots*. We will not use most of these, but here are some of them.

In [4]:
kaili_occs_df.columns

Index(['occurrence_no', 'record_type', 'reid_no', 'flags', 'collection_no',
       'identified_name', 'identified_rank', 'identified_no', 'difference',
       'accepted_name',
       ...
       'ecospace_comments', 'composition', 'architecture', 'thickness',
       'reinforcement', 'genus.1', 'formation.1', 'stratgroup.1', 'member.1',
       'primary_reference'],
      dtype='object', length=123)

Now we want to get a list of just each taxon, which happens to be conveniently put in the "identified_name" column. This basically collapses all occurrences into a single taxon.

In [5]:
kaili_id_name_set = set()
for id_name in kaili_occs_df['identified_name']:
    kaili_id_name_set.add(id_name)
burgess_id_name_set = set()
for id_name in burgess_occs_df['identified_name']:
    burgess_id_name_set.add(id_name)

poleta_id_name_set = set()
for id_name in poleta_occs_df['identified_name']:
    poleta_id_name_set.add(id_name)

# Print how many in each.
print(f'Kaili taxa: {len(kaili_id_name_set)}')
print(f'Poleta taxa: {len(poleta_id_name_set)}')
print(f'Burgess Shale taxa: {len(burgess_id_name_set)}')

Kaili taxa: 130
Poleta taxa: 45
Burgess Shale taxa: 279


Now let's dump the taxonomic info for every one of these into a CSV file, along with references.

First Kaili formation...


In [6]:
with open("kaili_pbdb_taxa.csv", "w") as outfile:
    print("phylum,class,order,family,genus,identified_name,references", file=outfile)
    for taxon in kaili_id_name_set:
        # First let's get a list of all the references for this name. We don't want to dump
        # lots of duplicate references, so we will use a set.
        refs = kaili_occs_df.primary_reference[kaili_occs_df.identified_name == taxon]
        ref_set = set()
        for ref in refs:
            ref_set.add(ref)

        phylum = kaili_occs_df[kaili_occs_df['identified_name'] == taxon].iloc[0]['phylum']
        tax_class = kaili_occs_df[kaili_occs_df['identified_name'] == taxon].iloc[0]['class']
        order = kaili_occs_df[kaili_occs_df['identified_name'] == taxon].iloc[0]['order']
        family = kaili_occs_df[kaili_occs_df['identified_name'] == taxon].iloc[0]['family']
        genus = kaili_occs_df[kaili_occs_df['identified_name'] == taxon].iloc[0]['genus']

        print(f'{phylum},{tax_class},{order},{family},{genus},{taxon},', file=outfile, end='')
        for ref in ref_set:
            # need to remove double quotes in titles.
            ref = ref.replace('"', "'")
            print(f'\"{ref}\",', file=outfile, end='')
        print('\n', file=outfile, end='')
print("Kaili done.")

Kaili done.


Poleta:

In [7]:
with open("poleta_pbdb_taxa.csv", "w") as outfile:
    print("phylum,class,order,family,genus,identified_name,references", file=outfile)
    for taxon in poleta_id_name_set:
        refs = poleta_occs_df.primary_reference[poleta_occs_df.identified_name == taxon]
        ref_set = set()
        for ref in refs:
            ref_set.add(ref)
        phylum = poleta_occs_df[poleta_occs_df['identified_name'] == taxon].iloc[0]['phylum']
        tax_class = poleta_occs_df[poleta_occs_df['identified_name'] == taxon].iloc[0]['class']
        order = poleta_occs_df[poleta_occs_df['identified_name'] == taxon].iloc[0]['order']
        family = poleta_occs_df[poleta_occs_df['identified_name'] == taxon].iloc[0]['family']
        genus = poleta_occs_df[poleta_occs_df['identified_name'] == taxon].iloc[0]['genus']

        print(f'{phylum},{tax_class},{order},{family},{genus},{taxon},', file=outfile, end='')
        for ref in ref_set:
            # need to remove double quotes in titles.
            ref = ref.replace('"', "'")
            print(f'\"{ref}\",', file=outfile, end='')
        print('\n', file=outfile, end='')
print("Poleta done.")

Poleta done.


Burgess shale:

In [8]:
with open("burgess_pbdb_taxa.csv", "w") as outfile:
    print("phylum,class,order,family,genus,identified_name,references", file=outfile)
    for taxon in burgess_id_name_set:
        refs = burgess_occs_df.primary_reference[burgess_occs_df.identified_name == taxon]
        ref_set = set()
        for ref in refs:
            ref_set.add(ref)
        phylum = burgess_occs_df[burgess_occs_df['identified_name'] == taxon].iloc[0]['phylum']
        tax_class = burgess_occs_df[burgess_occs_df['identified_name'] == taxon].iloc[0]['class']
        order = burgess_occs_df[burgess_occs_df['identified_name'] == taxon].iloc[0]['order']
        family = burgess_occs_df[burgess_occs_df['identified_name'] == taxon].iloc[0]['family']
        genus = burgess_occs_df[burgess_occs_df['identified_name'] == taxon].iloc[0]['genus']

        print(f'{phylum},{tax_class},{order},{family},{genus},{taxon},', file=outfile, end='')
        for ref in ref_set:
            # need to remove double quotes in titles.
            ref = ref.replace('"', "'")
            print(f'\"{ref}\",', file=outfile, end='')
        print('\n', file=outfile, end='')
print("Burgess done.")

Burgess done.
